In [1]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json
# ------- STUFF WE MIGHT NEED LATER ----------
# import matplotlib.pyplot as plt
# import gmaps
import urllib
import time
from datetime import date
# import seaborn as sbn
# from scipy import stats


# # Gmaps API Keys
# from config import (gkey)

# # Configure gmaps
# gmaps.configure(api_key=gkey)

# Creating an array of ranges to iterate through all the API calls
arr = np.arange(0, 178000, 2000)
arr

array([     0,   2000,   4000,   6000,   8000,  10000,  12000,  14000,
        16000,  18000,  20000,  22000,  24000,  26000,  28000,  30000,
        32000,  34000,  36000,  38000,  40000,  42000,  44000,  46000,
        48000,  50000,  52000,  54000,  56000,  58000,  60000,  62000,
        64000,  66000,  68000,  70000,  72000,  74000,  76000,  78000,
        80000,  82000,  84000,  86000,  88000,  90000,  92000,  94000,
        96000,  98000, 100000, 102000, 104000, 106000, 108000, 110000,
       112000, 114000, 116000, 118000, 120000, 122000, 124000, 126000,
       128000, 130000, 132000, 134000, 136000, 138000, 140000, 142000,
       144000, 146000, 148000, 150000, 152000, 154000, 156000, 158000,
       160000, 162000, 164000, 166000, 168000, 170000, 172000, 174000,
       176000])

In [24]:
#This API request somehow limits the number of records returned to 2000. But we can download all of them from the site
# as a json file. However, if we want we can iterate through the 178000 records 89 times and get them all :)

base_url = "https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/Fire_Incidents_Public/FeatureServer/0/query?"

field_list = "OBJECTID,incident_number,incident_type,incident_type_description,arrive_date_time,dispatch_date_time,exposure,platoon,station,address,address2,apt_room,GlobalID,cleared_date_time"
parameters = f"where=1%3D1&outFields={field_list}&outSR=4326&f=json"

set_length = 1

# resultsOffset is the record to start with, resultRecordCount is the number of records to return, max=2000
#For iterating through API calls to get all the data. We know there are 177,738 Rows, so we need to run this 89 times.

records = f'&resultOffset={set_length}&resultRecordCount=2000'

# fire_data = requests.get(base_url + parameters + records).json()


In [3]:
# Looks like we can iterate through the calls to get all the records.
# Here is a function that iterates through a list and returns all the JSON data to a hard file

def arcgis_api(sets):
    fire_data = []
    # Work with the log file to record the API calls/errors
    timestr = time.strftime('%Y%m%d-%H%M%S')
    log = 'log_' + timestr + '.txt'
    print(f'Logging API calls in {log}')

    with open(log, 'a+') as f:

        for n in sets:
            status = (f'Getting data for records {n}: {n+2000}... ')
            print(status)
            #Perform the API call on ARCGIS
            try:
                url = base_url + parameters + f'&resultOffset={n}&resultRecordCount=2000'
                records = requests.get(url).json()
                f.write(url)
                fire_data.append(records)
            #Log the result
                msg = 'Success!\n'
                f.write(msg)
                print(msg)
                time.sleep(2)
            except requests.exceptions.ContentDecodingError as e:
                msg = f'There was an error with the set starting with record {n}\n'
                f.write(msg)
                print(msg)

    with open('fire' + timestr + '.txt', 'a+') as file:
        print(f'Dumping json data to {file}')
        json.dump(fire_data, file)



In [5]:
# Calling the function with the array created above
# that will do the 89 API calls and save all the data to the fire_timestamp_txt file
# There will also be a logfile of the transactions
#COMMENTING THIS OUT SO IT DOESN'T RUN AGAIN
# arcgis_api(arr)


In [3]:
# START HERE!!!
firefile = '122000.json'
with open(firefile) as jsonfile:
    fire_data = json.load(jsonfile)
    
fire_data['features'][0]['attributes']

{'OBJECTID': 594765,
 'incident_number': '15-0022476',
 'incident_type': 611,
 'incident_type_description': 'Dispatched & cancelled en route',
 'arrive_date_time': None,
 'dispatch_date_time': 1437760020000,
 'exposure': 0,
 'platoon': 'B',
 'station': 2,
 'address': '911 ILEAGNES RD RALEIGH, NC 27601',
 'address2': 'PEACH RD COMM CTR                       ',
 'apt_room': '               ',
 'GlobalID': '273f1785-39de-4855-98a6-894f59de6fdd',
 'cleared_date_time': 1437760200000}

In [4]:
#Once we have all the json information we can parse them out as such
incidents = []
lat = []
lon = []
# I'm stuck here: The call data for each
# for sets in fire_data:
for i in fire_data['features']:
    try:
        incidents.append(i['attributes'])
        lat.append(i['geometry']['y'])
        lon.append(i['geometry']['x'])
    except:
        lat.append('missing')
        lon.append('missing')
#             print('Missing Geometry... ')
len(incidents)


2000

In [5]:
df = pd.DataFrame(incidents)

In [6]:
df.head()

,GlobalID,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station
0,273f1785-39de-4855-98a6-894f59de6fdd,594765,"911 ILEAGNES RD RALEIGH, NC 27601",PEACH RD COMM CTR,,NaN,1437760200000,1437760020000,0,15-0022476,611.0,Dispatched & cancelled en route,B,2.0
1,491f1208-9f3c-4902-84a9-89f0e062a3bb,594766,"311 STONEY MOSS DR RALEIGH, NC 27601",,110,1.438397e+12,1438399140000,1438397220000,0,15-0023269,743.0,Smoke detector activation no fire - unintentional,B,12.0
2,12b1f572-df8a-45b1-a85e-7314c27d0b6d,594767,"510 W JOHNSON ST RALEIGH, NC 27601",,,1.436382e+12,1436382000000,1436381580000,0,15-0020778,743.0,Smoke detector activation no fire - unintentional,A,1.0
3,004c7a5f-a1a1-4b61-96ce-3dd2685b5a00,594768,"2901 TRAWICK RD RALEIGH, NC 27601",UNITED SKATES OF AMERICA,,1.436532e+12,1436532540000,1436532060000,0,15-0020961,735.0,Alarm system sounded due to malfunction,A,11.0
4,1dd3cc20-aa6b-4ce8-9c92-053b2f66c19c,594769,"3616 POOLE RD RALEIGH, NC 27601",BP GAS RA,,1.437240e+12,1437241740000,1437239940000,0,15-0021810,671.0,HazMat release investigation w/no HazMat,C,12.0


In [7]:
df['Longitude'] = lon
df['Latitude'] = lat

In [8]:
df.dtypes

GlobalID                      object
OBJECTID                       int64
address                       object
address2                      object
apt_room                      object
arrive_date_time             float64
cleared_date_time              int64
dispatch_date_time             int64
exposure                       int64
incident_number               object
incident_type                float64
incident_type_description     object
platoon                       object
station                      float64
Longitude                     object
Latitude                      object
dtype: object

In [9]:
df.head()

,GlobalID,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station,Longitude,Latitude
0,273f1785-39de-4855-98a6-894f59de6fdd,594765,"911 ILEAGNES RD RALEIGH, NC 27601",PEACH RD COMM CTR,,NaN,1437760200000,1437760020000,0,15-0022476,611.0,Dispatched & cancelled en route,B,2.0,-78.6551,35.7427
1,491f1208-9f3c-4902-84a9-89f0e062a3bb,594766,"311 STONEY MOSS DR RALEIGH, NC 27601",,110,1.438397e+12,1438399140000,1438397220000,0,15-0023269,743.0,Smoke detector activation no fire - unintentional,B,12.0,-78.5813,35.7766
2,12b1f572-df8a-45b1-a85e-7314c27d0b6d,594767,"510 W JOHNSON ST RALEIGH, NC 27601",,,1.436382e+12,1436382000000,1436381580000,0,15-0020778,743.0,Smoke detector activation no fire - unintentional,A,1.0,-78.6465,35.7873
3,004c7a5f-a1a1-4b61-96ce-3dd2685b5a00,594768,"2901 TRAWICK RD RALEIGH, NC 27601",UNITED SKATES OF AMERICA,,1.436532e+12,1436532540000,1436532060000,0,15-0020961,735.0,Alarm system sounded due to malfunction,A,11.0,-78.5894,35.8197
4,1dd3cc20-aa6b-4ce8-9c92-053b2f66c19c,594769,"3616 POOLE RD RALEIGH, NC 27601",BP GAS RA,,1.437240e+12,1437241740000,1437239940000,0,15-0021810,671.0,HazMat release investigation w/no HazMat,C,12.0,-78.5766,35.7678


In [10]:
df = df.loc[df['Longitude']!='missing',:]

In [11]:
len(df)

1976

In [12]:
df = df.loc[df['incident_type_description']!='NULL',:]

In [13]:
len(df)

1927

In [15]:
df = df.dropna()
len(df)

1509

In [16]:
df['dispatch_date_time'] = df['dispatch_date_time']/1000
df.dtypes

GlobalID                      object
OBJECTID                       int64
address                       object
address2                      object
apt_room                      object
arrive_date_time             float64
cleared_date_time              int64
dispatch_date_time           float64
exposure                       int64
incident_number               object
incident_type                float64
incident_type_description     object
platoon                       object
station                      float64
Longitude                     object
Latitude                      object
dtype: object

In [17]:
df.head()

,GlobalID,OBJECTID,address,address2,apt_room,arrive_date_time,cleared_date_time,dispatch_date_time,exposure,incident_number,incident_type,incident_type_description,platoon,station,Longitude,Latitude
1,491f1208-9f3c-4902-84a9-89f0e062a3bb,594766,"311 STONEY MOSS DR RALEIGH, NC 27601",,110,1.438397e+12,1438399140000,1.438397e+09,0,15-0023269,743.0,Smoke detector activation no fire - unintentional,B,12.0,-78.5813,35.7766
2,12b1f572-df8a-45b1-a85e-7314c27d0b6d,594767,"510 W JOHNSON ST RALEIGH, NC 27601",,,1.436382e+12,1436382000000,1.436382e+09,0,15-0020778,743.0,Smoke detector activation no fire - unintentional,A,1.0,-78.6465,35.7873
3,004c7a5f-a1a1-4b61-96ce-3dd2685b5a00,594768,"2901 TRAWICK RD RALEIGH, NC 27601",UNITED SKATES OF AMERICA,,1.436532e+12,1436532540000,1.436532e+09,0,15-0020961,735.0,Alarm system sounded due to malfunction,A,11.0,-78.5894,35.8197
4,1dd3cc20-aa6b-4ce8-9c92-053b2f66c19c,594769,"3616 POOLE RD RALEIGH, NC 27601",BP GAS RA,,1.437240e+12,1437241740000,1.437240e+09,0,15-0021810,671.0,HazMat release investigation w/no HazMat,C,12.0,-78.5766,35.7678
5,8e50a1b4-4485-4a87-b838-26327b51b5bf,594770,"101 DURHAM CO WAY RALEIGH, NC 27601",DCDH,,1.436391e+12,1436391000000,1.436390e+09,0,15-9018104,611.0,Dispatched & cancelled en route,A,24.0,-78.6345,35.7755


In [114]:
# df_sorted = df.sort_values('incident_type')

In [115]:
# df_sorted.groupby('incident_type').count()

In [18]:
# Adding the float format gets rid of all the ugly errors when trying to convert dates!
df.to_csv("output/extra_cleaned_fire_data.csv", index=False, header=True, float_format='%.f')